In [ ]:
'''

%cd /content/drive/MyDrive/'Aria Workshop'/food
import os
files = list(set([f.removesuffix('.jpg').removesuffix('.jpg.json') for f in os.listdir() if os.path.isfile(f)]))
N=len(files)
# randomly split f_names into two lists using np.random
# save the two lists into two files

import random
import shutil
random.shuffle(files)

person_a = files[:N//2]
person_b = files[N//2:]



# move data from src to dest
for i in person_a:
      dest = '../person_a/food'
      jpg= '{}.jpg'.format(i)
      json = '{}.jpg.json'.format(i)
      shutil.move(jpg, dest)
      shutil.move(json, dest)


for i in person_b:
      dest = '../person_b/food'
      jpg= '{}.jpg'.format(i)
      json = '{}.jpg.json'.format(i)
      shutil.move(jpg, dest)
      shutil.move(json, dest)''''''

In [ ]:
%cd /content

/content


In [ ]:
!pip install langchain openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
'''import csv
import json
import pandas as pd

# Function to convert a CSV to JSON
# Takes the file paths as arguments
def make_json(csvFilePath, jsonFilePath):

    # create a dictionary
    data = {}

    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)

        # Convert each row into a dictionary
        # and add it to data
        for rows in csvReader:

            # Assuming a column named 'No' to
            # be the primary key
            key = rows['track']
            data[key] = rows

    # Open a json writer, and use the json.dumps()
    # function to dump data
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))

# Driver Code

# Decide the two file paths according to your
# computer system
csvFilePath = '/content/drive/MyDrive/Aria Workshop/person_a/person_a_streaming.csv'
jsonFilePath = '/content/drive/MyDrive/Aria Workshop/person_a/person_a_streaming.json'


# Call the make_json function
make_json(csvFilePath, jsonFilePath)'''

In [ ]:
# load jsons
import json
from os.path import join
def load_json(path):
  with open(path) as f:
      return json.load(f)

dir = '/content/drive/MyDrive/Aria Workshop/'
files = ['artwork', 'food', 'map', 'streaming', 'meta']
raw_person1 = [load_json(join(dir, 'person_a', 'person_a_'+i+'.json')) for i in files]
raw_person2 = [load_json(join(dir, 'person_b', 'person_b_'+i+'.json')) for i in files]


In [ ]:
data_person1 = [' '.join(map(str, data)) for data in raw_person1]
data_person2  = [' '.join(map(str, data)) for data in raw_person2]

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from langchain.document_loaders.csv_loader import CSVLoader
from langchain import PromptTemplate
from langchain.schema import HumanMessage, SystemMessage
import os
from tqdm.auto import tqdm

# user calls kill_switch whenever a trigger phrase is heard to kill the process of sharing timeline
killed = False

def kill_switch():
	killed = True

def load_chain(model, p_template):
	PROMPT = PromptTemplate(template=p_template, input_variables=["text"])
	chain = load_summarize_chain(model, chain_type="stuff", prompt=PROMPT)
	return chain

summary_prompt_template = """
Write a concise summary (between 5 to 10 sentences) of the following text in json format.
Please keep in mind that the format is in json, but I need the summary to be free-form text.
Please take note of key details and the categories they belong to. Add it to the summary.
Use statements like "the user likes" to allow for future comparision.


{text}

TL'DR:
"""

card_prompt_template = """ Transform the provided data into a personalized personality card.
The entries should be in list form.
It should contain important information about the person, and what are their preferences.

{text}

"""

compare_subprompt = '''Compare the two provided personality cards named Card 1 and Card 2, and '''

similarities_prompt_template = compare_subprompt + """suggest topics based on their similarities. {text} """
differences_prompt_template = compare_subprompt + """suggest topics based on their differences. {text} """
generate_prompt_template = compare_subprompt + """generate questions based on their cards related to travel, food, music. {text} """

os.environ["OPENAI_API_KEY"] = ""

llm = ChatOpenAI(temperature=0, model_name="gpt-4")
summarize_chain = load_chain(llm, summary_prompt_template)
card_chain = load_chain(llm, card_prompt_template)
sim_chain = load_chain(llm, similarities_prompt_template)
dif_chain = load_chain(llm, differences_prompt_template)
gen_questions_chain = load_chain(llm, generate_prompt_template)


if not killed:
	# create summary
	summary_person1 = [files[i]+ ": " + summarize_chain.run([Document(page_content=data_person1[i])]) for i in tqdm(range(len(data_person1)))]
	summary_person2 = [files[i]+ ": " + summarize_chain.run([Document(page_content=data_person2[i])]) for i in tqdm(range(len(data_person2)))]


In [ ]:
summary_person1

["artwork: The user likes a variety of art styles and artists. They have shown interest in Impressionism, specifically Claude Monet's 'Water Lilies' and 'The Water Lily Pond'. They also like Cubism, with works from Kazimir Malevich and Georges Braque. Malevich's 'Portrait of a Polish Woman' and 'Three Figures' are among their favorites. They also appreciate Abstract art, including works by Wassily Kandinsky. Other styles they enjoy include Expressionism, Fauvism, Modernism, Symbolism, Surrealism, and Post-Impressionism. Notable artists include Henri Matisse, Pablo Picasso, Bernardino Luini, Marc Chagall, Frida Kahlo, Boris Kustodiev, Zinaida Serebriakova, Juan Gris, and Louise Bourgeois.",
 'food: The text provides a list of dishes from various cuisines, their ingredients, and cooking methods. For instance, Japanese cuisine includes dishes like Steamed Rice and Chicken Oyakodon, cooked by steaming and simmering respectively. Italian cuisine features dishes like Spaghetti Carbonara and 

In [ ]:
if not killed:
  # reduce into card
  card_person1 = card_chain.run([Document(page_content=' '.join(map(str, summary_person1)))])
  card_person2 = card_chain.run([Document(page_content=' '.join(map(str, summary_person2)))])

  cards = f"Card 1: {card_person1} and Card 2: {card_person2}"

  # compare
  gen_answers = gen_questions_chain.run([Document(page_content=' '.join(map(str, cards)))])

In [ ]:
print(card_person1)

Personality Card:

- Art Preferences: 
  - Impressionism: Claude Monet's 'Water Lilies' and 'The Water Lily Pond'
  - Cubism: Kazimir Malevich's 'Portrait of a Polish Woman' and 'Three Figures', Georges Braque
  - Abstract Art: Wassily Kandinsky
  - Other Styles: Expressionism, Fauvism, Modernism, Symbolism, Surrealism, Post-Impressionism
  - Notable Artists: Henri Matisse, Pablo Picasso, Bernardino Luini, Marc Chagall, Frida Kahlo, Boris Kustodiev, Zinaida Serebriakova, Juan Gris, Louise Bourgeois

- Food Preferences:
  - Japanese: Steamed Rice, Chicken Oyakodon
  - Italian: Spaghetti Carbonara, Spaghetti Bolognese
  - French: Lobster Bisque
  - Chinese: Hot and Sour Soup, Fried Rice
  - American: Clam Chowder, Lobster Bisque

- Travel History:
  - Hamburg Hauptbahnhof, Germany
  - Moana Surfrider and St. Catalina Seaside Chapel, Honolulu, Hawaii
  - San Francisco International Airport and Golden Gate Bridge, California
  - Roissy-en-France, France
  - Various locations in Copenhagen,

In [ ]:
print(card_person2)

Personality Card:

- Art Preferences: 
  - Impressionism: Claude Monet's "Water Lilies" and "The Water Lily Pond"
  - Cubism: "Portrait of a Polish Woman" by Kazimir Malevich and "Violin and Pitcher" by Georges Braque
  - Abstract Art: Works by Kazimir Malevich and Wassily Kandinsky
  - Expressionism: "Black Lines" by Wassily Kandinsky and "Park von Saint-Cloud" by Oskar Kokoschka
  - Other Styles: Suprematism, Realism, Modernism, Symbolism, Fauvism, and Surrealism

- Food Preferences:
  - Italian: Spaghetti Carbonara, Spaghetti Bolognese, and Fettuccine Alfredo
  - Chinese: Fried Rice and Hot and Sour Soup
  - Japanese: Udon and Miso Soup
  - American: Clam Chowder and Lobster Bisque
  - Cooking Methods: Boiling, sautéing, baking, frying, simmering, and steaming

- Travel History (April-August 2023):
  - Germany: Hamburg Hauptbahnhof
  - United States: Moana Surfrider, St. Catalina Seaside Chapel, Makapu’u Lighthouse Road, Waikiki Beachside Hostel, Kamehameha Highway, San Francisco In

In [ ]:
generate_prompt_template = compare_subprompt + """generate questions based on their cards related to travel, food, and music. {text} """
gen_questions_chain = load_chain(llm, generate_prompt_template)
gen_answers = gen_questions_chain.run([Document(page_content=' '.join(map(str, cards)))])
print(gen_answers)

1. Both users seem to have a preference for Japanese, French, Italian, Chinese, and American cuisines. Can you share some of your favorite dishes from each of these cuisines?
2. User 1 has a trip planned to Hamburg Hauptbahnhof, Germany in April 2023. What are some of the attractions or activities you are looking forward to during this trip?
3. User 2 has a broad interest in art, including Surrealism, Fauvism, Modernist, Symbolist, Post-Impressionism, and Realism. Can you share some of your favorite pieces or artists from these styles?
4. Both users have a trip planned to Hawaii in August 2023. What are some of the activities or attractions you are most excited about for this trip?
5. User 1 has a diverse taste in music, with songs like "Free Britney", "How Would They Know", and "Candy from Strangers". Can you share why these songs resonate with you?
6. User 2 also has a diverse taste in music, with songs like "All American Nightmare", "Wherever I May Roam", and "Genocidal Humanoidz". 

In [ ]:
with open('/content/drive/MyDrive/Aria Workshop/person_a/card.txt', 'w') as f:
  f.write(card_person1)

with open('/content/drive/MyDrive/Aria Workshop/person_b/card.txt', 'w') as f:
  f.write(card_person2)

with open('/content/drive/MyDrive/Aria Workshop/icebreakers.txt', 'w') as f:
  f.write(gen_answers)

In [ ]:
'''
BACKUP prompt
def finalize(data, prompt_template):
	messages = [
    SystemMessage(
        content="You are a helpful assistant that is great at identifying important details from the data."
    ),
    HumanMessage(
        content=prompt_template+data
    )
		]
	return llm(messages)'''